In [498]:
import pandas as pd
import numpy as np
from collections import defaultdict
import random
import itertools

In [499]:
random.seed(23)

In [639]:
def generate_rand_matrix(n,m):
    rand_mat = np.random.randint(2, size=(n,m))

    return rand_mat  

def get_neighbors(mat):
    neighbor_dict = defaultdict(list)
    n,m =mat.shape[0],mat.shape[1]
    for i in range(n):
        for j in range(m):
            x = [[i+y[0], j+y[1], 0] for y in itertools.product([1,0,-1], [1,0,-1])]
            z = [e for e in x if (0 <= e[0] < n and 0 <= e[1] < m and e[:2] != [i, j])]
            neighbor_dict[(i, j)] = z

    for k,v in neighbor_dict.items():
        for e in v:
            if mat[e[0],e[1]]==1:
                e[2]=1
            else:
                e[2]=0

    return neighbor_dict


def next_gen(mat):
    neighbor_sum_dict = defaultdict(int)
    mat_copy = mat.copy()
    neighbors = get_neighbors(mat)

    for k,v in neighbors.items():
        neighbors_sum = sum([e[2] for e in v])
        
        neighbor_sum_dict[k] = neighbors_sum
        
    for k,v in neighbor_sum_dict.items():
        i, j = k

        if (v < 2 or v > 3) and mat[i, j] == 1:
            mat_copy[i, j] = 0

        elif mat[i, j] == 0 and v == 3:
            mat_copy[i, j] = 1
            
    return mat_copy


def run_game(mat, max_steps=5000):
    step = 0
    steady_count = 0  

    while step < max_steps:
        new_mat = next_gen(mat)

        if np.array_equal(new_mat, mat):
            steady_count += 1
        else:
            steady_count = 0 

        if steady_count == 5:
            print(f"Steady state reached at step {step}.")
            break
        
        if np.all(new_mat == 0):
            print(f"All cells are dead at step {step}.")
            break

        mat = new_mat  
        step += 1

    return mat,new_mat


In [652]:
def indvidual_matrix_data_collection(mat):
    alive_cells = mat.sum()
    total_cells = mat.size


    return alive_cells, total_cells


def run_data_collection():
    print("x")

    

In [650]:
test_matrix = generate_rand_matrix(100,100)

In [651]:
indvidual_matrix_data_collection(test_matrix)

(np.int64(4970), 10000)

In [ ]:
test_matrix.sum()


np.int64(24)

In [642]:
run_game(test_matrix)

Steady state reached at step 7.


(array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 1, 1]], dtype=int32),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 1, 1]], dtype=int32))